<a href="https://colab.research.google.com/github/Lis30/Homeworks/blob/main/%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BA%D0%B0_5_1_%D0%9A%D0%B0%D1%80%D0%BF%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

1) Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:<br>
InvoiceNo – номер транзакции<br>
StockCode – код товара<br>
Description – описание товара<br>
Quantity – количество единиц товара, добавленных в заказ<br>
InvoiceDate – дата транзакции <br>
UnitPrice – цена за единицу товара<br>
CustomerID – id клиента<br>
Country – страна, где проживает клиент<br>
Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [2]:
retail = pd.read_csv('/content/data.csv.zip', compression='zip', encoding='ISO-8859-1')

In [3]:
retail_columns = retail.columns

In [5]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

2) Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [6]:
print(f'количество повторов в таблице - {retail.duplicated().sum()}')

количество повторов в таблице - 5268


In [7]:
retail = retail.drop_duplicates() #удаляем дубликаты

3) ⭐️Задание со звёздочкой!⭐️<br>
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). <br>
Сколько всего заказов отменили пользователи?

In [8]:
print(f"Отменённых заказов {retail.InvoiceNo.apply(lambda x: x.startswith('C')).sum()}")

Отменённых заказов 9251


4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [11]:
retail.query('Quantity > 0').info()

<class 'pandas.core.frame.DataFrame'>
Index: 526054 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    526054 non-null  object 
 1   StockCode    526054 non-null  object 
 2   Description  525462 non-null  object 
 3   Quantity     526054 non-null  int64  
 4   InvoiceDate  526054 non-null  object 
 5   UnitPrice    526054 non-null  float64
 6   CustomerID   392732 non-null  float64
 7   Country      526054 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.1+ MB


5) 💀 Сложное задание! 💀<br>
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [22]:
germany_top = retail.query('Country == "Germany"').groupby('CustomerID', as_index=False).agg({'InvoiceNo': 'count'})
#за процентиль отвечает функция quantile
quant = germany_top.InvoiceNo.quantile(q=0.8)
quant

164.0

In [27]:
germany_top = germany_top.query('InvoiceNo > @quant').CustomerID

In [30]:
germany_top = germany_top.reset_index().CustomerID

In [32]:
germany_top.shape

(19,)

6) ⭐️Задание со звёздочкой!⭐️<br>
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [42]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]

7) Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? <br>
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [49]:
top_retail_germany.groupby('StockCode', as_index=False).agg({'InvoiceNo': 'count'}).sort_values('InvoiceNo', ascending=False).head(3)

,StockCode,InvoiceNo
1299,POST,185
464,22326,75
514,22423,57


8) Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [51]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [52]:
retail.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


9) Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)<br>

Например, для следующего примера...<br>

InvoiceNo StockCode Quantity InvoiceDate          UnitPrice  CustomerID  Revenue<br>
536365....85123A....6.....2010-12-01..08:26:00..2.55.....17850.0.....15.30<br>
536365....71053.....6.....2010-12-01..08:26:00..3.39.....17850.0.....20.34<br>
555555....71053.....1.....2010-12-02..18:00:00..3.39.....17850.0.....3.39<br>
...сумма заказа будет равна:<br>

InvoiceNo  TotalRevenue<br>
536365.....35.64<br>
555555.....3.39<br>

И ответом будет:<br>
536365, 555555

In [56]:
retail.groupby('InvoiceNo').agg({'Revenue': 'sum'}).sort_values('Revenue', ascending=False).head(5)

,Revenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00


In [55]:
retail[~retail.InvoiceNo.str.startswith('C')].query('Revenue < 0')
#увидела, что есть минуса в сумме чека, отфильтровала "С"шки (отменённые). Два чека продали в большой минус

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom,-11062.06
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom,-11062.06
